<a href="https://colab.research.google.com/github/asmaalqadi/data-feature/blob/main/Acticity1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Assignment 1:


We will build a data feature that tells people news about the teams in a match and league of their choice.

We will use two APIs:


1. An API for football matches
2. An API to find the news relating to the teams

Steps:


*   Sign up for a football API key
*   Sign up for a news API key
*   Add API Key's to colab's secrets
*   Test the APIs
*   Function to get the uocoming matches data about the Leauges
*   Function to get the article data for the the teams chosen









In [ ]:
#importing both requests and json
import requests
import json

# initializing  the keys for the APIs
from google.colab import userdata
FOOTBALL_API_KEY = userdata.get('FOOTBALL_API_KEY')
NEWS_API_KEY = userdata.get('NEWS_API_KEY')



In [ ]:
import requests
# creating the functions for league selection
def league_selector():
#  Condition that the user choice should be one of these options
    while True:
        print("Select the league to view upcoming matches:")
        print("1. Champions League")
        print("2. Premier League")
        print("3. French League")
        print("4. Italian League")
        print("5. German League")
        print("6. Spanish League")
        league_choice = input("Enter your League choice (1-6): ")
# Dictionary with the choices to limit the user data entry
# All codes below are according the league ID in the football API
        league_ids = {
            # code for the Champions League
            '1': 2,
            # code for the Premier League
            '2': 39,
            # code for the French League
            '3': 61,
            # code for the Italian League
            '4': 135,
            # code for the German League
            '5': 78,
            # code for the Spanish League
            '6': 140}
# if league_choice in ID's then return the corresponding league
        if league_choice in league_ids:
            return league_ids[league_choice]
        else:
# if league_choice was out of the options retun this error message and allow re-entry
            print("Invalid choice, please enter a number from the options above.")



In [ ]:
# creating the function for getting the upcoming 10 matches for a certain league based on the date the code is ran with a parameter of the league_id
def upcoming_matches(league_id):
  # the url for the football API added to fetch this data from it
    url = "https://api-football-v1.p.rapidapi.com/v3/fixtures"
    headers = {
        'x-rapidapi-key': FOOTBALL_API_KEY,
        'x-rapidapi-host': 'api-football-v1.p.rapidapi.com'
    }
    # parameters including the league_id accorrding to API and the number of matches fetched
    params = {
        'league': league_id,
        'next': 10
    }
    response = requests.get(url , headers=headers , params=params)
    # The success number 200 as per the API
    if response.status_code == 200:
        matches = response.json()
        return matches['response']
    else:
        print(f"Failed to fetch matches: {response.status_code} - {response.text}")
        return None


In [ ]:
def match_selector():
  # getting the list of matches from the football API
    matches = upcoming_matches()
    # if there are 10 matches available to be displayed then they will available for the user's selection
    if matches:
      # the user is then asked to pick a match from (1-10)
        print("Select a match by entering its number (1-10):")
        # numbering the matches by this loop from 1-10 using the enumerate function
        for i, match in enumerate(matches, 1):
            # some formats for displaying the teams names'
            home_team = match['teams']['home']['name']
            away_team = match['teams']['away']['name']
            # Date format to match UTC
            match_date = match['fixture']['date'].replace("T", " ").split("+")[0]
            # formatting the match fixture to be displayed with the date using f- string
            print(f"{i}. {home_team} vs {away_team} on {match_date}")

        league_choice = int(input("Enter your choice (number): "))
        # checks if the number is entered from the list
        if 1 <= league_choice <= len(matches):
          #league choice is subtractted by 1 to match the dictionary since the numbering starts from 1
            return matches[league_choice - 1]
        else:
            print("Invalid choice. Please try again.")
            return None
    # if not in list this message will be printed
    else:
        print("No upcoming matches available.")
        return None

In [ ]:
def news_articles(team_name):
   # gets the URL for NY Times article by team name using the news API
    url = f"https://api.nytimes.com/svc/search/v2/articlesearch.json?q={team_name} football"
    params = {'api-key':NEWS_API_KEY}
    response = requests.get(url, params=params)
    # if successful then 200 , according to API
    if response.status_code == 200:
      # extract articles
        return response.json()['response']['docs']
    else:
        print(f"Failed to fetch news articles for {team_name}: {response.status_code}") # error message
        return None


In [ ]:
def teams_news(home_team, away_team):
  # getting home and away team news
    home_news = news_articles(home_team)
    away_news = news_articles(away_team)
  # Home team news printed first
    print(f"News for {home_team}:")
    for article in home_news:
        print(f"- {article['headline']['main']} (URL: {article['web_url']})")
# Away team news then is printed
    print(f"News for {away_team}:")
    for article in away_news:
        print(f"- {article['headline']['main']} (URL: {article['web_url']})")

# user is asked for a league id
league_id = league_selector()
# checks if the user number is valid
if league_id:
  # prints the list of upcoming chosen league matches using the ID entered
    matches = upcoming_matches(league_id)
    if matches:
        print("Upcoming Matches:")
        # printing the list using enumerate function
        for index, match in enumerate(matches, 1):
            home_team = match['teams']['home']['name']
            away_team = match['teams']['away']['name']
            match_date = match['fixture']['date'].replace("T", " ").split("+")[0]
            print(f"{index}. {home_team} vs {away_team} on {match_date} UTC")

        # asking the user for a match from the upcoming matches choices
        # the int is used to change the number entered by the user and then subtract it by 1
        selected_index = int(input("Enter the match number to view news about the teams: ")) - 1
        # checking if the choice is valid
        if 0 <= selected_index < len(matches):
            selected_match = matches[selected_index]
            teams_news(selected_match['teams']['home']['name'], selected_match['teams']['away']['name'])
            # error messages if the number is invalid
        else:
            print("Invalid match selection.")
    else:
        print("No upcoming matches found for the selected league.")
else:
    print("No valid league was selected, please restart the program.")

Select the league to view upcoming matches:
1. Champions League
2. Premier League
3. French League
4. Italian League
5. German League
6. Spanish League
